In [ ]:
from preprocess import *
from utils import *
from datagen import *
from model import *
from model import *

In [ ]:
news_index, news_title, word_dict, news_senti, news_senti_cate = process_news(['train','valid'])
train_candidate,train_user_his,train_label = process_users('train',news_index,True)
test_candidate,test_user_his,test_label,test_index = process_users('valid',news_index,False)
emb_table = embedding('glove.840B.300d.txt',word_dict)

In [ ]:
model_vec, gan, senti_pred, user_model, news_encoder = senti_model(word_dict, emb_table)

In [ ]:
max_steps=50000
for ep in range(1):
    train_gen = generate_batch_data_train(news_title,news_senti_cate,train_candidate,train_user_his,train_label,30)
    iterations=max_steps
    d_loss=0.
    a_loss=0.
    for step,data in enumerate(train_gen):
        disvec1,disvec2=model_vec.predict([data[0][0],data[0][1]])
        senti_pred.trainable = True
        d_loss =senti_pred.train_on_batch([disvec1,disvec2],[to_categorical(data[1][2][:,:,0],5),to_categorical(data[1][3][:,:,0],5)],[data[2][2],data[2][3]])
        senti_pred.trainable = False
        a_loss =gan.train_on_batch(data[0],[data[1][0],data[1][1],to_categorical(data[1][2][:,:,0],5),to_categorical(data[1][3][:,:,0],5)],data[2])
        step+=1
        d_loss+=np.array(d_loss)
        a_loss+=np.array(a_loss)

        if step%100==0:
            print(step,d_loss/step,a_loss/step)
            
        if step==iterations:
            break
            
        if step%10000==0:
            usermodel = Model([news_input], userrep)
            test_gen=generate_batch_data_user(news_title, test_user_his, 100) 
            user_vec=user_model.predict_generator(test_gen,steps=len(test_user_his)//100+1,verbose=1)
            news_vec=news_encoder.predict([news_title],batch_size=200,verbose=1)

            for i in range(len(test_index)):
                newscand=news_vec[test_candidate[test_index[i][0]:test_index[i][1]]]
                uvector=user_vec[i]
                scores=1/(1+np.exp(-np.tensordot(uvector,newscand,axes=(0,1))))
                m=test_index[i]
                if m[1]-m[0]>1:
                    all_auc.append(auc(test_label[m[0]:m[1]],scores))
                    all_mrr.append(mrr_score(test_label[m[0]:m[1]],scores))
                    all_ndcg.append(ndcg_score(test_label[m[0]:m[1]],scores,k=5))
                    all_ndcg_2.append(ndcg_score(test_label[m[0]:m[1]],scores,k=10))
                    if len(all_auc)%10000==0:
                        print(len(all_auc)) 
                        
            print(np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg_2))
            
            senti_top=[]
            for i in range(len(test_index)):
                newscand=news_vec[test_candidate[test_index[i][0]:test_index[i][1]]]
                uvector=user_vec[i]
                scores=1/(1+np.exp(-np.tensordot(uvector,newscand,axes=(0,1))))
                ranks=np.argsort(1-scores)
            
                senti_top.append([np.mean(news_senti[test_candidate[test_index[i][0]:test_index[i][1]][ranks[:x]]]) for x in range(1,100)])
                if i%10000==0:
                    print(i)
                    print(np.mean(senti_top,axis=0))